#Long-Short Equity

In [0]:
#if you are using google drive like me
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd "/content/drive/My Drive/go_beyond/project/project_1"

/content/drive/My Drive/go_beyond/project/project_1


In [0]:
#helper file with functions created in stocks returns analysis
%run helper_returns.ipynb

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import math

#remove pandas warning 
import pandas.plotting._converter as pandacnv
pandacnv.register()

##Loading Data 

You can see the analysis of this data in stocks returns file. :)

All funcitions have been explained [here](https://https://nbviewer.jupyter.org/github/victormenuzzo/Quantitative-Analysis/blob/master/part1/stocks_returns.ipynb)

In [0]:
df = pd.read_csv('eod-quotemedia.csv', parse_dates=['date'], index_col=False)
close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')

#resample daily -> monthly
df_monthly = resample_prices(close)

#raws returns
df_rawRet = compute_rawReturn(df_monthly)

#logs returns 
df_logRet = compute_logReturn(df_monthly)

##Trading Signal

First of all, we need to know what is a trading signal. And [investopedia](https://www.investopedia.com/terms/t/trade-signal.asp) will explain to us: P


>A trade signal is a trigger for action, either to buy or sell a security or other asset, generated by analysis. That analysis can be human generated using technical indicators, or it can be generated using mathematical algorithms based on market action, possibly in combination with other market factors such as economic indicators.


---


We will create a simple trading signal: 
For each month, rank the stocks from the highest to the lowest. 
* Top performing stocks -> Long portfolio
* Bottom performing stocks -> Short portfolio


In [0]:
def top_stocks(df, n):
  df_top = df.copy()
  df_top[:] = 0

  for date, month_return in df.iterrows():
    #nlargest: Return the first n rows ordered by columns in descending order.
    df_top.loc[date, month_return.nlargest(n).index] = 1
  return df_top

In [0]:
#this is our top performing stocks
df_top = top_stocks(df_logRet, 3)

#if we multiply df_monthly by -1 we get bottom performing
df_bottom = top_stocks(df_logRet*-1, 3)

Now we can see our Long and Short portfolio

In [0]:
def show_LongShort(df, n):
  return df.sum().sort_values(ascending=False).index[:n].values.tolist()

In [68]:
print('Top Performing: ')
print(show_LongShort(df_top, 10))

print(20*'-')

print('Bottom Performing: ')
print(show_LongShort(df_bottom, 10))

Top Performing: 
['FCX', 'NFLX', 'AMD', 'MU', 'TRIP', 'EA', 'INCY', 'QRVO', 'COTY', 'SIG']
--------------------
Bottom Performing: 
['CHK', 'FCX', 'NEM', 'WYNN', 'SPLS', 'UAA', 'PXD', 'QRVO', 'M', 'PRGO']


## Checking
This strategy only serves to exemplify a trading signal as it presents some problems. Let's check! :D

In [79]:
print('Number of months: {0}\nNumber of stocks: {1}\n'.format(str(df_top.shape[0]), str(df_top.shape[1])))

Number of months: 47
Number of stocks: 495


In [0]:
#next steps